#### 1- Import Required Libraries ??

In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

c:\Users\hp\anaconda3\envs\llamaindex-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 2- Load The Data ??

In [2]:
data = SimpleDirectoryReader("data")

documents = data.load_data()

In [3]:
documents

[Document(id_='e09e0585-a360-43bb-9b06-7023f5096f59', embedding=None, metadata={'page_label': '1', 'file_name': 'Basics_of_QC.pdf', 'file_path': 'd:\\Artifical Intelligence World\\3. LLMs\\Workshops\\YouTube\\Code Commander\\1. Complete Gen AI Series\\LlamaIndex\\data\\Basics_of_QC.pdf', 'file_type': 'application/pdf', 'file_size': 353084, 'creation_date': '2025-07-27', 'last_modified_date': '2025-07-27'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=' \nA Study on the basics of Quantum Computing \n \n \n    Prashant \n \nDepartment d’Informatique et de recherché operationnelle, \nUniversite de Montreal, Montreal. Canada. \n             

In [4]:
len(documents)

42

Prepare Simple Input Prompt ?

In [5]:
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."

## This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["HUGGINGFACE_API_TOKEN"] = os.getenv("HUGGINGFACE_API_TOKEN")


#### 3- Load the Mistral LLM From HuggingFace ??

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

#### 4- Load Also The Embedding Model From HuggingFace ??

In [8]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from llama_index.core import Settings
from llama_index.embeddings.langchain import LangchainEmbedding

In [ ]:
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2"
    )
)

#### 5- Mistral With LlamaIndex Service Context -> Settings ??

In [ ]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024


#### 6- Q/A ??

In [ ]:
index = VectorIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()

response = query_engine.query("What is the AutoEncoder in the area of Deep Learning?")


In [ ]:
print(response)

#### 6- Display the Response ??

In [ ]:
from IPython.display import Markdown, display

display(Markdown(f"<b>{response}</b>"))